In [14]:
import torchtext
import torch
import torch.nn as nn
import torch.optim as optim

import time
import random

## 8.4.1 IMDb 데이터를 읽고 데이터 로더 작성(BERT의 Tokenizer사용)
- 첫째, 단어 분할 Tokenizer에 BERT용 Tokenizer를 사용합니다. 7장에서는 공백으로 구분하는 함수를 직접 만들었찌만 이번에는 앞서 구현한 BertTokenizer 클래스의 tokenize 함수를 사용합니다.
- 둘째, torchtext에서 데이터 로더를 작성할 때 vocabulary인 TEXT.vocab을 만드는 방법이 다릅니다. 7장에서는 훈련 데이터에 포함된 단어로 vocabulary를 작성했습니다. BERT는 미리 준비된 vocab 폴더 내 bert-base-uncased-vocab.txt의 30,522단어(정확하게는 서브워드)를 모두 사용한 vocabulary를 만듭니다. BERT의 모든 단어를 사용하여 BertEmbeddings모듈을 작성하기 때문입니다.
- 두 가지 차이점에 주의하면서 구현합니다.
- 먼저 문장의 전처리와 단어 분할을 묶은 tokenizer_with_preprocessing 함수를 구현합니다.

In [2]:
# 전처리 및 단어 분할을 묶은 함수 작성
import re
import string
from utils.bert import BertTokenizer
# utils 폴더의 bert.py를 불러들인다.

def preprocessing_text(text):
    '''IMDb 전처리'''
    # 개행 코드 삭제
    text = re.sub('<br />', '', text)
    
    # 쉼표, 마침표 외의 기호를 공백(스페이스)으로 대체
    for p in string.punctuation:
        if (p == '.') or (p == ','):
            continue
        else:
            text = text.replace(p, ' ')
    
    # 마침표 등의 전후에 공백을 넣는다
    text = text.replace('.', ' . ')
    text = text.replace(',', ' , ')
    return text

# 단어 분할용 Tokenizer 준비
tokenizer_bert = BertTokenizer(
    vocab_file='./vocab/bert-base-uncased-vocab.txt', do_lower_case=True)

# 전처리와 단어 분할을 묶은 함수 정의
# 단어 분할 함수를 전달하므로 tokenizer_bert대신 tokenizer_bert.tokenize를 전달하는 점에 주의
def tokenizer_with_preprocessing(text, tokenizer=tokenizer_bert.tokenize):
    text = preprocessing_text(text)
    ret = tokenizer(text) # tokenizer_bert
    return ret

In [ ]:
# 데이터를 읽었을 때 내용에 수행할 처리 정의
max_length = 256

TEXT = torchtext.data.Field(sequential=True,
                           tokenize=tokenizer_with_preprocessing, use_vocab=True,
                           lower=True, include_lengths=True, batch_first=True,
                           fix_length=max_length, init_token='[CLS]',
                           eos_token='[SEP]', pad_token='[PAD]',
                           unk_token='[UNK]')
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

# 각 인수 재확인
# sequential: 데이터의 길이가 달라질 수 있는가? 문장은 길이가 다양하므로 True, 라벨은 False
# tokenize: 문장을 읽을 때 전처리 및 단어 분할 함수 정의
# use_vocab: vocabulary에 단어를 추가할지 여부
# lower: 알파벳일 있을 때 소문자로 변환할지 여부
# include_length: 문장의 단어 수 데이터를 포함할지 여부
# batch_first: 미니 배치 차원을 선두에 제공할지 여부
# fix_length: 전체 문장을 지정한 길이가 되도록 padding
# init_token, eos_token, pad_token, unk_token : 문장 선두, 문장 말미, padding, 미지어에 어떠한 단어를 부여하는 지정

In [15]:
# data 폴더에서 각 tsv파일을 읽는다.
# BERT용으로 처리하므로 10분 정도 시간이 걸린다.
train_val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path = './data/', train='IMDb_train.tsv',
    test ='IMDb_test.tsv', foramt ='tsv',
    fields=[('Text', TEXT), ('Label',LABEL)])

# torchtext.data.Dataset의 split함수로 훈련 데이터와 검증 데이터를 나눈다.
train_ds, val_ds = train_val_ds.split(
    split_ratio=0.8, random_state=random.seed(1234))

AttributeError: module 'torchtext.data' has no attribute 'TabularDataset'

In [ ]:
# BERT는 BERT가 가진 모든 단어로 BertEmbedding모듈을 작성하여 vocabulary는 전체 단어를 사용한다.
# 훈련 데이터로 vocabulary를 만들지 않는다.

# 우선 BERT용의 단어 사전을 사전형 변수에 준비한다.
from utils.bert import BertTokenizer, load_vocab

vocab_bert, ids_to_tokens_bert = load_vocab(
    vocab_file = './vocab/bert-base-uncased-vocab.txt')

# TEXT.vocab.stoi = vocab_bert(stoi는 string_to_ID로 단어에서 ID로 사전)로 하고 싶지만 
# build_vocab를 실행하지 않으면 텍스트 오브젝트가 vocab의 멤버 변수를 갖지 않는다.
# ('Field' object has no attribute 'vocab' 오류 발생)

# 적당히 build_vocab에서 vocabulary를 작성하고 BERT의 vocabulary를 덮어 쓴다.
TEXT.build_vocab(train_ds, min_freq=1)
TEXT.vocab.stoi = vocab_bert

In [ ]:
# 데이터 로더 작성(torchtext에서 iterator라고 불린다)
batch_size = 32 # BERT에서는 16, 32 근처를 사용

train_dl = torchtext.data.Iterator(
    train_ds, batch_size=batch_size, train=True)

val_ds = torchtext.data.Iterator(
    val_ds, batch_size=batch_size, train=False, sort=False)

test_ds = torchtext.data.Iterator(
    test_ds, batch_size=batch_size, train=False, sort=False)

# 사전 객체로 정리
dataloaders_dict = {'train':train_dl, 'val':val_dl}

In [ ]:
# 동작 확인 검증 데이터셋으로 확인
batch = next(iter(val_dl))
print(batch.Text)
print(batch.Label)

## 8.4.2 감정 분석용 BERT 모델 구축
- BERT모델에 대해 학습된 파라미터를 읽어들이고 긍정적인지 부정적인지 분류하는 어댑터 모듈을 설치하여 감정 분석을 하는 BERT 모델을 구축합니다.
- BERT의 기본 모델을 구축한 후 학습된 파라미터를 읽어들입니다.

In [3]:
from utils.bert import get_config, BertModel, set_learned_params

# 모델 설정 JSON파일을 오브젝트 변수로 가져온다
config = get_config(file_path='./weights/bert_config.json')

# BERT 모델 작성
net_bert = BertModel(config)

# BERT 모델에 학습된 파라미터 설정
net_bert = set_learned_params(
    net_bert, weights_path='./weights/pytorch_model.bin')

bert.embeddings.word_embeddings.weight→embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight→embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight→embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.gamma→embeddings.LayerNorm.gamma
bert.embeddings.LayerNorm.beta→embeddings.LayerNorm.beta
bert.encoder.layer.0.attention.self.query.weight→encoder.layer.0.attention.selfattn.query.weight
bert.encoder.layer.0.attention.self.query.bias→encoder.layer.0.attention.selfattn.query.bias
bert.encoder.layer.0.attention.self.key.weight→encoder.layer.0.attention.selfattn.key.weight
bert.encoder.layer.0.attention.self.key.bias→encoder.layer.0.attention.selfattn.key.bias
bert.encoder.layer.0.attention.self.value.weight→encoder.layer.0.attention.selfattn.value.weight
bert.encoder.layer.0.attention.self.value.bias→encoder.layer.0.attention.selfattn.value.bias
bert.encoder.layer.0.attention.output.dense.weight→encoder.layer.0.attention.output

- BERT의 기본 모델에 문장 분류를 위한 어댑터로 전결합 층을 하나만 연결한 BertForIMDb 클래스 생성
- BERT는 클래스 분류 시 문장 첫 번째 단어 [CLS]의 특징량을 입력한 텍스트 데이터의 특징량을 사용합니다.
- BERT는 선두 단어의 특징량을 사용하여 Next Sentence Prediction을 사전 학습 작업으로 실행. BERT에서 입력 문장의 의미를 알 수 있는(적어도 입력된 두 문장의 의미가 연결되었는지 판단이 설 정도의 정보를 보유하는)것 처럼 선두 단어의 특징량을 만드는 방법이 사전 학습되었으며 선두 단어의 특징량이 입력 문장 전체의 특징을 반영합니다. 반면에 Transformer는 사전 작업이 없습니다.
- 문장 분류에 선두 단어를 사용하여 역전파로 선두 단어의 특징량이 텍스트 데이터의 특징을 나타내도록 학습. BERT는 선두 단어가 입력 테스트 전체의 특징을 가지도록 사전 작업에서 결합 파라미터가 학습

In [5]:
import torch.nn as nn

class BertForIMDb(nn.Module):
    '''BERT 모델에 IMDb 내용이 긍정적/부정적인지 판정하는 부분을 연결한 모델'''
    
    def __init__(self, net_bert):
        super(BertForIMDb, self).__init__()
        
        # BERT 모듈
        self.bert = net_bert # BERT 모델
        
        # head에 긍정적/부정적 예측 추가
        # 입력은 BERT의 출력 특징량의 차원, 출력은 긍정적/부정적 두 가지
        self.cls = nn.Linear(in_features=768, out_features=2)
        
        # 가중치 초기화 처리
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)
        
    def forward(self, input_ids, token_type_ids=None, attention_mask=None, output_all_encoded_layers=False, attention_show_fig=False):
        '''
        input_ids: [batch_size, sequence_length] 문장의 단어 ID 나열
        token_type_ids: [batch_size, sequence_length] 각 단어가 첫 번째 문장인지 두 번째 문장인지 나타내는 id
        attention_mask: Transformer의 마스크와 같은 기능의 마스킹
        output_all_encoded_layers: 반환 값을 전체 TransformerBlock 모듈의 출력으로 할 것인지 마지막 층만 한정할지의 플래그
        attention_show_fig: Self-Attention의 가중치를 반환할지의 플래그
        '''
        
        # BERT의 기본 모델 부분의 순전파
        # 순전파한다.
        if attention_show_fig == True:
            '''attention_show의 경우 attention_probs도 반환'''
            encoded_layers, pooled_output, attention_probs = self.bert(
                input_ids, token_type_ids, attention_mask,
                output_all_encoded_layers, attention_show_fig)
        elif attention_show_fig == False:
            encoded_layers, pooled_output = self.bert(
                input_ids, token_type_ids, attention_mask,
                output_all_encoded_layers, attention_show_fig)
            
        # 입력 문장의 첫 단어 [CLS]의 특징량을 사용하여 긍정적/부정적인지 분류
        vec_0 = encoded_layers[:, 0, :]
        vec_0 = vec_0.view(-1, 768) # size를 (batch_size, hidden_size)로 변환
        out = self.cls(vec_0)
        
        # attention_show의 경우 attention_probs(마지막)도 반환
        if attention_show_fig == True:
            return out, attention_probs
        elif attention_show_fig == False:
            return out

In [6]:
# 모델 구축
net = BertForIMDb(net_bert)

# 훈련 모드로 설정
net.train()

print('네트워크 설정 완료')

네트워크 설정 완료


## 8.4.3 BERT의 파인튜닝을 위한 설정
- BERT관련 논문에서는 12단 BertLayer의 모든 파라미터를 튜닝합니다.
- 12단 모두 파인튜닝하기에는 많은 시간이 필요(+ GPU 메모리를 많이 차지하여 미니 배치 크기를 32에서 16으로 해야함)
- 학습 시간을 단축하기 위하여 마지막 12단의 BertLayer만 파인 튜닝하여 1~11단의 BertLayer 파라미터는 변경하지 않도록 설정

In [8]:
# 기울기 계산을 마지막 BertLayer 모듈과 추가한 분류 어댑터만 실행

# 1. 먼저 모두 기울기 계산 False로 한다.
for name, param in net.named_parameters():
    param.requires_grad = False

# 2. 마지막 BertLayer 모듈을 기울기 계산하도록 변경
for name, param in net.bert.encoder.layer[-1].named_parameters():
    param.requires_grad = True

# 3. 식별기를 기울기 계산을 하도록 변경
for name, param in net.cls.named_parameters():
    param.requires_grad = True

- 계속하여 최적화 기법과 손실함수 정의. 최적화 설정은 BERT 논문에서 권장된 파라미터를 사용

In [10]:
# 최적화 기법 설정

# BERT의 원래 부분을 파인튜닝
optimizer = optim.Adam([
    {'params':net.bert.encoder.layer[-1].parameters(), 'lr':5e-5},
    {'params':net.cls.parameters(), 'lr':5e-5}
], betas=(0.9, 0.999))

# 손실함수 설정
criterion = nn.CrossEntropyLoss()
# nn.LogSoftmax()를 계산한 후 nn.NLLoss(negative log likelihood loss) 계산

## 8.4.4 학습 및 검증 실시
- BertForIMDb의 학습 및 검증을 실행.
- 이번 절에서는 [PAD]에 대해 Self-Attention을 적용하지 않도록 하는 attention_mask를 생략하여 None으로 진행
- 사전 학습으로 [PAD]가 의미 없음을 배웠고 이번 절 내용은 [PAD]에 대한 attention_mask를 생략해도 성능에는 거의 변함이 없음

In [12]:
#모델을 학습시키는 함수 작성
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    
    # GPU를 사용할 수 있는지 확인
    device = torch.deivce('mps' if torch.backends.mps.is_available() else 'cpu')
    print('사용장치:', device)
    print('----start----')
    
    # 네트워크를 GPU로
    net.to(device)
    
    # 네트워크가 어느 정도 고정되면 고속화시킨다.
    torch.backends.cudnn.benchmark = True
    
    # 미니 배치 크기
    batch_size = dataloaders_dict['train'].batch_size
    
    # 에폭 루프
    for epoch in range(num_epochs):
        # 에폭별 훈련 및 검증 루프
        for phase in ['train','val']:
            if phase == 'train':
                net.train() # 모델을 훈련 모드로
            else:
                net.eval()  # 모델을 검증 모드로
                
            epoch_loss = 0.0 # 에폭의 손실 합
            epoch_corrects = 0 # 에폭의 정답 수
            iteration = 1
            
            # 개시 시간 저장
            t_epoch_start = time.time()
            t_iter_start = time.time()
            
            # 데이터 로더에서 미니 배치를 꺼내는 루프
            for batch in (dataloaders_dict[phase]):
                # batch는 텍스트와 라벨의 사전형 변수
                
                # GPU를 사용할 수 있다면 GPU로 데이터를 보낸다
                inputs = batch.Text[0].to(device) # 문장
                labels = batch.Label.to(device)  # 라벨
                
                # 옵티마이저 초기화
                optimizer.zero_grad()
                
                # 순전파 계산
                with torch.set_grad_enabled(phase=='train'):
                    
                    # BertForIMDb에 입력
                    outputs = net(inputs, token_type_ids=None, attention_mask=None,
                                  output_all_encoded_layers=False, attention_show_fig=False)
                    
                    loss = criterion(outputs, labels) # 손실 계산
                    
                    _, preds = torch.max(outputs, 1) # 라벨 예측
                    
                    # 훈련 시에는 역전파
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                        if (iteration % 10 == 0): # 10iter에 한 번 손실을 표시
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            acc = (torch.sum(preds == labels.data)
                                  ).double()/batch_size
                            print('반복 {} || Loss: {:.4f} || 10iter: {:.4f} sec. || 이 반복의 정답률: {}'.format(
                                    iteration, loss.item(), duration, acc))
                            t_iter_start = time.time()
                            
                    iteration += 1
                    
                    # 손실과 정답 수의 합계 갱신
                    epoch_loss += loss.item() * batch_size
                    epoch_corrects += torch.sum(preds == labels.data)
                    
            # 에폭별 손실과 정답률
            t_epoch_finish = time.time()
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)
            
            print('Epoch {}/{} | {:^5} | Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs, phase, epoch_loss, epoch_acc))
            t_epoch_start = time.time()
            
    return net

num_epochs = 2
net_trained = train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

NameError: name 'dataloaders_dict' is not defined

In [ ]:
# 학습한 네트워크 파라미터 저장
save_path = './weights/bert_fine_tuning_IMDb.pth'
torch.save(net_trained.state_dict(), save_path)

# 테스트 데이터의 정답률을 구한다.
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

net_trained.eval() # 모델을 검증 모드로
net_trained.to(device)

# 에폭의 정답 수를 기록하는 변수
epoch_corrects = 0

for batch in tqdm(test_dl): # 테스트 데이터의 데이터 로더
    # batch는 텍스트와 라벨의 사전 오브젝트
    # GPU를 사용할 수 있다면 GPU로 데이터를 보낸다.
    device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
    inputs = batch['Text'][0].to(device) # 문장
    labels = batch.Label.to(device)      # 라벨
    
    # 순전파 계산
    with torch.set_grad_enabled(False):
        
        # BertForIMDb에 입력
        outputs = net_trained(inputs, token_type_ids=None, attention_mask=None,
                              output_all_encoded_layers=False, attention_show_fig=False)
        
        loss = criterion(outputs, labels) # 손실 계산
        _, preds = torch.max(outputs, 1) # 라벨 예측
        epoch_corrects += torch.sum(preds == labels.data) # 정답 수의 합계 갱신
        
# 정답률
epoch_acc = epoch_corrects.double() / len(test_dl.dataset)

print('테스트 데이터 {}개에서 정답률 : {:.4f}'.format(len(test_dl.dataset), epoch_acc))